In [3]:
!pip3 install torch torchvision
!pip3 install safetensors

  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)


In [15]:
model = mobilenet_v2(weights="MobileNet_V2_Weights.IMAGENET1K_V1")
print(model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [19]:
model = mobilenet_v2(weights="MobileNet_V2_Weights.IMAGENET1K_V1")
for name, module in model.named_modules():
    if hasattr(module, 'weight') and hasattr(module.weight, 'shape'):
        print(f"{name}: {module.weight.shape}") # [out_channels, in_channels, kH, kW]

features.0.0: torch.Size([32, 3, 3, 3])
features.0.1: torch.Size([32])
features.1.conv.0.0: torch.Size([32, 1, 3, 3])
features.1.conv.0.1: torch.Size([32])
features.1.conv.1: torch.Size([16, 32, 1, 1])
features.1.conv.2: torch.Size([16])
features.2.conv.0.0: torch.Size([96, 16, 1, 1])
features.2.conv.0.1: torch.Size([96])
features.2.conv.1.0: torch.Size([96, 1, 3, 3])
features.2.conv.1.1: torch.Size([96])
features.2.conv.2: torch.Size([24, 96, 1, 1])
features.2.conv.3: torch.Size([24])
features.3.conv.0.0: torch.Size([144, 24, 1, 1])
features.3.conv.0.1: torch.Size([144])
features.3.conv.1.0: torch.Size([144, 1, 3, 3])
features.3.conv.1.1: torch.Size([144])
features.3.conv.2: torch.Size([24, 144, 1, 1])
features.3.conv.3: torch.Size([24])
features.4.conv.0.0: torch.Size([144, 24, 1, 1])
features.4.conv.0.1: torch.Size([144])
features.4.conv.1.0: torch.Size([144, 1, 3, 3])
features.4.conv.1.1: torch.Size([144])
features.4.conv.2: torch.Size([32, 144, 1, 1])
features.4.conv.3: torch.Size

In [1]:
import json
import torch
from torchvision.models import mobilenet_v2

model = mobilenet_v2(weights="MobileNet_V2_Weights.IMAGENET1K_V1")
model.eval()

param_dict = {}

# 1. Save weights and biases (Conv, Linear, BN weight, BN bias)
for name, param in model.named_parameters():
    param_dict[name] = param.detach().cpu().numpy().flatten().tolist()

# 2. Save buffers (BN running_mean, running_var, etc.)
for name, buf in model.named_buffers():
    # e.g. "features.1.1.running_mean"
    param_dict[name] = buf.detach().cpu().numpy().flatten().tolist()

# 3. Save eps of BatchNorm layers
# (eps is not a buffer, so need to save it manually)
for name, module in model.named_modules():
    if isinstance(module, torch.nn.BatchNorm2d):
        param_dict[name + ".eps"] = module.eps

with open("mobilenet_v2_full.json", "w") as f:
    json.dump(param_dict, f)


In [14]:
# Load json and print layer names and shapes
with open("mobilenet_v2_full.json", "r") as f:
    param_dict = json.load(f)

for name, values in param_dict.items():
    if isinstance(values, list):
        print(f"{name}: {len(values)}")
    else:
        print(f"{name}: {values} (scalar)")

features.0.0.weight: 864
features.0.1.weight: 32
features.0.1.bias: 32
features.1.conv.0.0.weight: 288
features.1.conv.0.1.weight: 32
features.1.conv.0.1.bias: 32
features.1.conv.1.weight: 512
features.1.conv.2.weight: 16
features.1.conv.2.bias: 16
features.2.conv.0.0.weight: 1536
features.2.conv.0.1.weight: 96
features.2.conv.0.1.bias: 96
features.2.conv.1.0.weight: 864
features.2.conv.1.1.weight: 96
features.2.conv.1.1.bias: 96
features.2.conv.2.weight: 2304
features.2.conv.3.weight: 24
features.2.conv.3.bias: 24
features.3.conv.0.0.weight: 3456
features.3.conv.0.1.weight: 144
features.3.conv.0.1.bias: 144
features.3.conv.1.0.weight: 1296
features.3.conv.1.1.weight: 144
features.3.conv.1.1.bias: 144
features.3.conv.2.weight: 3456
features.3.conv.3.weight: 24
features.3.conv.3.bias: 24
features.4.conv.0.0.weight: 3456
features.4.conv.0.1.weight: 144
features.4.conv.0.1.bias: 144
features.4.conv.1.0.weight: 1296
features.4.conv.1.1.weight: 144
features.4.conv.1.1.bias: 144
features.4.c